In [1]:
import numpy as np
from numpy.random import default_rng
import pandas as pd
from make_standings import Standings
from make_charts import make_playoff_charts, make_win_charts
from itertools import permutations
import time
import base64

In [2]:
df = pd.read_csv("schedules/schedule21.csv")
pr_both = pd.read_csv("data/pr_both.csv",squeeze=True,index_col="Side")
div_series = pd.read_csv("data/divisions.csv",squeeze=True,index_col=0)
teams = sorted(list(set(df.team_home)))
conf_teams = {}
for conf in ["AFC","NFC"]:
    conf_teams[conf] = [t for t in teams if div_series[t][:3]==conf]
rng = np.random.default_rng()

In [3]:
"ARI_Off" in pr_both.keys()

True

In [4]:
pr_both.keys()

Index(['ARI_Off', 'ATL_Off', 'BAL_Off', 'BUF_Off', 'CAR_Off', 'CHI_Off',
       'CIN_Off', 'CLE_Off', 'DAL_Off', 'DEN_Off', 'DET_Off', 'GB_Off',
       'HOU_Off', 'IND_Off', 'JAX_Off', 'KC_Off', 'LAC_Off', 'LAR_Off',
       'LVR_Off', 'MIA_Off', 'MIN_Off', 'NE_Off', 'NO_Off', 'NYG_Off',
       'NYJ_Off', 'PHI_Off', 'PIT_Off', 'SEA_Off', 'SF_Off', 'TB_Off',
       'TEN_Off', 'WAS_Off', 'ARI_Def', 'ATL_Def', 'BAL_Def', 'BUF_Def',
       'CAR_Def', 'CHI_Def', 'CIN_Def', 'CLE_Def', 'DAL_Def', 'DEN_Def',
       'DET_Def', 'GB_Def', 'HOU_Def', 'IND_Def', 'JAX_Def', 'KC_Def',
       'LAC_Def', 'LAR_Def', 'LVR_Def', 'MIA_Def', 'MIN_Def', 'NE_Def',
       'NO_Def', 'NYG_Def', 'NYJ_Def', 'PHI_Def', 'PIT_Def', 'SEA_Def',
       'SF_Def', 'TB_Def', 'TEN_Def', 'WAS_Def', 'HFA', 'mean_score'],
      dtype='object', name='Side')

In [5]:
div_series

ARI     NFC West
ATL    NFC South
BAL    AFC North
BUF     AFC East
CAR    NFC South
CHI    NFC North
CIN    AFC North
CLE    AFC North
DAL     NFC East
DEN     AFC West
DET    NFC North
GB     NFC North
HOU    AFC South
IND    AFC South
JAX    AFC South
KC      AFC West
LAC     AFC West
LAR     NFC West
LVR     AFC West
MIA     AFC East
MIN    NFC North
NE      AFC East
NO     NFC South
NYG     NFC East
NYJ     AFC East
PHI     NFC East
PIT    AFC North
SEA     NFC West
SF      NFC West
TB     NFC South
TEN    AFC South
WAS     NFC East
Name: 0, dtype: object

In [6]:
def simulate_reg_season(df):
    df.loc[:,["score_home","score_away"]] = rng.normal(df[["mean_home","mean_away"]],10)
    df.loc[:,["score_home","score_away"]] = df.loc[:,["score_home","score_away"]].astype(int)
    adjust_ties(df)
    return None

def adjust_ties(df):
    tied_games = df.loc[df.score_home == df.score_away,["score_home","score_away"]].copy()
    x = rng.normal(size=len(tied_games))
    tied_games.iloc[np.where((x > -1.6) & (x < 0))[0]] += (0,3)
    tied_games.iloc[np.where(x >= 0)[0]] += (3,0)
    df.loc[tied_games.index,["score_home","score_away"]] = tied_games
    return None

In [7]:
df["pr_home_Off"] = df.team_home.map(lambda s: pr_both[s+"_Off"])
df["pr_home_Def"] = df.team_home.map(lambda s: pr_both[s+"_Def"])
df["pr_away_Off"] = df.team_away.map(lambda s: pr_both[s+"_Off"])
df["pr_away_Def"] = df.team_away.map(lambda s: pr_both[s+"_Def"])
df["mean_home"] = df["pr_home_Off"]-df["pr_away_Def"]+pr_both["mean_score"]+pr_both["HFA"]/2
df["mean_away"] = df["pr_away_Off"]-df["pr_home_Def"]+pr_both["mean_score"]-pr_both["HFA"]/2

In [8]:
total_dict = {}

win_dict = {t:{i:0 for i in range(18)} for t in teams}
for conf in ["AFC","NFC"]:
    total_dict[conf] = {i:{t:0 for t in sorted(conf_teams[conf])} for i in range(1,8)}

In [9]:
div_dict = {}

s = div_series
for a,b in s.groupby(s):
    div_dict[a] = list(b.index)

rank_dict = {div:{} for div in div_dict.keys()}

for div in rank_dict.keys():
    for team_sort in permutations(div_dict[div]):
        rank_dict[div][team_sort] = 0

In [10]:
reps = 30

start = time.time()

for i in range(reps):
    simulate_reg_season(df)
    stand = Standings(df)
    s = stand.standings
    s_dict = dict(zip(s.index,s.Wins))
    for t,w in s_dict.items():
        win_dict[t][w] += 1
    for d in rank_dict.keys():
        rank_dict[d][tuple(stand.div_ranks[d])] += 1
    p = stand.playoffs
    for conf in ["AFC","NFC"]:
        for j,t in enumerate(p[conf]):
            total_dict[conf][j+1][t] += 1

end = time.time()

In [11]:
stand.div_ranks

{'AFC East': ['BUF', 'MIA', 'NE', 'NYJ'],
 'AFC North': ['CLE', 'BAL', 'PIT', 'CIN'],
 'AFC South': ['IND', 'HOU', 'JAX', 'TEN'],
 'AFC West': ['LAC', 'KC', 'DEN', 'LVR'],
 'NFC East': ['DAL', 'WAS', 'NYG', 'PHI'],
 'NFC North': ['MIN', 'CHI', 'DET', 'GB'],
 'NFC South': ['TB', 'CAR', 'NO', 'ATL'],
 'NFC West': ['LAR', 'SF', 'ARI', 'SEA']}

In [12]:

            
for div in stand.div_ranks.keys():
            rank_dict[div][tuple(stand.div_ranks[div])] += 1

In [13]:
rank_dict

{'AFC East': {('BUF', 'MIA', 'NE', 'NYJ'): 7,
  ('BUF', 'MIA', 'NYJ', 'NE'): 1,
  ('BUF', 'NE', 'MIA', 'NYJ'): 3,
  ('BUF', 'NE', 'NYJ', 'MIA'): 8,
  ('BUF', 'NYJ', 'MIA', 'NE'): 1,
  ('BUF', 'NYJ', 'NE', 'MIA'): 1,
  ('MIA', 'BUF', 'NE', 'NYJ'): 1,
  ('MIA', 'BUF', 'NYJ', 'NE'): 0,
  ('MIA', 'NE', 'BUF', 'NYJ'): 2,
  ('MIA', 'NE', 'NYJ', 'BUF'): 0,
  ('MIA', 'NYJ', 'BUF', 'NE'): 2,
  ('MIA', 'NYJ', 'NE', 'BUF'): 0,
  ('NE', 'BUF', 'MIA', 'NYJ'): 3,
  ('NE', 'BUF', 'NYJ', 'MIA'): 0,
  ('NE', 'MIA', 'BUF', 'NYJ'): 0,
  ('NE', 'MIA', 'NYJ', 'BUF'): 0,
  ('NE', 'NYJ', 'BUF', 'MIA'): 2,
  ('NE', 'NYJ', 'MIA', 'BUF'): 0,
  ('NYJ', 'BUF', 'MIA', 'NE'): 0,
  ('NYJ', 'BUF', 'NE', 'MIA'): 0,
  ('NYJ', 'MIA', 'BUF', 'NE'): 0,
  ('NYJ', 'MIA', 'NE', 'BUF'): 0,
  ('NYJ', 'NE', 'BUF', 'MIA'): 0,
  ('NYJ', 'NE', 'MIA', 'BUF'): 0},
 'AFC North': {('BAL', 'CIN', 'CLE', 'PIT'): 1,
  ('BAL', 'CIN', 'PIT', 'CLE'): 2,
  ('BAL', 'CLE', 'CIN', 'PIT'): 2,
  ('BAL', 'CLE', 'PIT', 'CIN'): 6,
  ('BAL', 'PIT', '

In [14]:
def prob_to_odds(p):
    if p < .000001:
        return "NA"
    if p > 0.5:
        x = 100*p/(p-1)
        return f"{x:.0f}"
    elif p <= 0.5:
        x = 100*(1-p)/p
        return f"+{x:.0f}"

In [18]:
stand.standings.loc["BUF","Wins"]

12